<a href="https://colab.research.google.com/github/AdamNizol/LivePortrait-jupyter/blob/main/LivePortrait_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LivePortrait
%cd /content/LivePortrait

!rm -rf /content/LivePortrait/pretrained_weights
!git clone https://huggingface.co/camenduru/LivePortrait /content/LivePortrait/pretrained_weights

!pip install tyro==0.8.5 onnxruntime-gpu==1.18.1 onnx==1.16.1 gradio==4.36.1 colorama

%cd /content/LivePortrait/src/utils/dependencies/insightface/thirdparty/face3d/mesh/cython
!python setup.py build_ext --inplace

%cd /content/LivePortrait

# Inference

In [ ]:
# !python inference.py -s /content/LivePortrait/assets/examples/source/m.jpeg -d /content/LivePortrait/assets/examples/driving/d3.mp4

# Define input image and driving video paths
input_image_path = "/content/LivePortrait/assets/examples/source/s7.jpg" #@param {type:"string"}

driving_video_path = "/content/LivePortrait/assets/examples/driving/d7.mp4" #@param {type:"string"}


# Extracting the image and video file names to generate the output file name
import os

image_filename = os.path.splitext(os.path.basename(input_image_path))[0]
video_filename = os.path.splitext(os.path.basename(driving_video_path))[0]
output_filename = f"{image_filename}--{video_filename}.mp4"
output_path = f"/content/LivePortrait/animations/{output_filename}"

# Run the inference using the defined paths
!python inference.py -s {input_image_path} -d {driving_video_path}

import subprocess

# Define the output path with audio
output_with_audio_filename = f"{image_filename}--{video_filename}_with_audio.mp4"
output_with_audio_path = f"/content/LivePortrait/animations/{output_with_audio_filename}"

# Use ffmpeg to extract audio from the original driving video and merge it with the generated video
subprocess.call([
    'ffmpeg', '-i', output_path, '-i', driving_video_path, '-c', 'copy', '-map', '0:v:0', '-map', '1:a:0', '-shortest', output_with_audio_path
])


# Display Result

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Read the output video file
mp4 = open(output_with_audio_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

# Display the video in HTML
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")
